In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import cartopy.feature as cfeature
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import matplotlib.pyplot as plt
import time
import importlib
import stormcoupling as sc
import glob
from scipy import stats
import scipy
from geopy import distance
from scipy import interpolate
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import cm
import xlrd
import csv
import os
import math
import matplotlib.colors
import metpy
from metpy.interpolate import interpolate_to_isosurface
from metpy.calc import dewpoint_from_specific_humidity
from metpy.calc import equivalent_potential_temperature
from matplotlib.ticker import LogFormatterSciNotation
import matplotlib.patheffects as pe
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from metpy.units import units 
import metpy.calc as mpcalc
importlib.reload(sc)

<module 'stormcoupling' from '/lustre06/project/6084782/shared/gabe-fall-2025/TEtracking/stormcoupling.py'>

In [ ]:
### formats data in a way that tempestextremes likes

In [14]:
geo = xr.open_dataset('data/era5-raw/geopotential_2015.nc')

In [5]:
geo.pressure_level

<xarray.DataArray 'pressure_level' (pressure_level: 3)> Size: 24B
array([700., 500., 250.])
Coordinates:
    number          int64 8B ...
  * pressure_level  (pressure_level) float64 24B 700.0 500.0 250.0
Attributes:
    long_name:         pressure
    units:             hPa
    positive:          down
    stored_direction:  decreasing
    standard_name:     air_pressure

In [15]:
geo = geo.rename({"valid_time": "time"})
geo["time"].encoding["units"] = "hours since 1900-01-01 00:00:00"
geo["time"].encoding["calendar"] = "gregorian"
geo_3hr = geo.sel(time=geo.time.dt.hour.isin([0, 3, 6, 9, 12,15, 18,21]))
geo_3hr = geo_3hr.sel(pressure_level=500)
#geo_3hr.to_netcdf("geopotential_subset_te_500.nc")

In [7]:
geo = geo.rename({"valid_time": "time"})
geo["time"].encoding["units"] = "hours since 1900-01-01 00:00:00"
geo["time"].encoding["calendar"] = "gregorian"
geo_3hr = geo.sel(time=geo.time.dt.hour.isin([0, 3, 6, 9, 12,15, 18,21]))
geo_3hr = geo_3hr.sel(pressure_level=250)
#geo_3hr.to_netcdf("geopotential_subset_te_250.nc")

In [13]:
geo = geo.rename({"valid_time": "time"})
geo["time"].encoding["units"] = "hours since 1900-01-01 00:00:00"
geo["time"].encoding["calendar"] = "gregorian"
geo_3hr = geo.sel(time=geo.time.dt.hour.isin([0, 3, 6, 9, 12,15, 18,21]))
geo_3hr = geo_3hr.sel(pressure_level=700)
geo_3hr.to_netcdf("geopotential_subset_te_700.nc")

In [20]:
geo_3hr.isel(time=0).z

<xarray.DataArray 'z' (latitude: 221, longitude: 581)> Size: 514kB
[128401 values with dtype=float32]
Coordinates:
    number          int64 8B ...
    time            datetime64[ns] 8B 2015-01-01
    pressure_level  float64 8B 500.0
  * latitude        (latitude) float64 2kB 75.0 74.75 74.5 ... 20.5 20.25 20.0
  * longitude       (longitude) float64 5kB -175.0 -174.8 ... -30.25 -30.0
    expver          <U4 16B ...
Attributes: (12/31)
    GRIB_paramId:                             129
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      128401
    GRIB_typeOfLevel:                         isobaricInhPa
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_shortName:                           z
    GRIB_totalNumber:                         0
    GRIB_units:                               m**2 s**-2
    long_name:                                Geopotential
    units:                                    m**2 s**-2
    standard_name:                            geopotential